In [228]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
import pickle
import keras_tuner as kt

In [28]:
Bank_data = pd.read_csv('Churn_Modelling.csv')
Bank_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [29]:
#Dropping all the columns unrelated to training the model
Bank_data = Bank_data.drop(['RowNumber','Surname','CustomerId'],axis=1)

In [30]:
Bank_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [32]:
#Label encoding the Gender column
Lb_encoder_Gen = LabelEncoder()
Bank_data['Gender']=Lb_encoder_Gen.fit_transform(Bank_data['Gender'])


In [33]:
Bank_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [35]:
#One hot encoding on Geography column
ohe_encoder_geo = OneHotEncoder()
geo_encoder = ohe_encoder_geo.fit_transform(Bank_data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [38]:
#Extracting names of the features totally availabel in geography after OHE_encoding
ohe_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [40]:
geo_encoder_df = pd.DataFrame(geo_encoder, columns=ohe_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [41]:
#Combing new encoded columns in Dataset and deleting the Geography column
Bank_data = pd.concat([Bank_data.drop('Geography',axis=1),geo_encoder_df],axis=1)
Bank_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [44]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(Lb_encoder_Gen,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(ohe_encoder_geo,file)

In [45]:
Bank_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [169]:
#Spiltting the dataset into dependent and independent features
X = Bank_data.drop('Exited',axis=1)
y = Bank_data['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [171]:
#Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [172]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [173]:
#saving the scaling
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [174]:
Bank_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [175]:
#ANN IMPLEMENTATION 

In [176]:
#the data is imbalanced
y.value_counts(normalize=True)

Exited
0    0.7963
1    0.2037
Name: proportion, dtype: float64

In [190]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Apply SMOTE
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Class distribution after SMOTE:", Counter(y_train_smote))

Class distribution after SMOTE: Counter({0: 6356, 1: 6356})


In [191]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Undersampling the majority class
undersample = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)

print("Class distribution after undersampling:", Counter(y_train_under))

Class distribution after undersampling: Counter({0: 6356, 1: 6356})


In [195]:
from imblearn.keras import BalancedBatchGenerator
from imblearn.under_sampling import NearMiss

# Create balanced batches
batch_generator = BalancedBatchGenerator(X_train, y_train, sampler=NearMiss(), batch_size=32, random_state=42)

In [196]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [197]:
(X_train.shape[1],)

(12,)

In [272]:
model = Sequential()

# First hidden layer
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Second hidden layer
model.add(Dense(80, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))


# Output layer
model.add(Dense(1, activation='sigmoid'))

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [273]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_25 (Dense)                │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 80)             │        10,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 80)             │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,897 (50.38 KB)

 Trainable params: 12,481 (48.75 KB)

 Non-trainable params: 416 (1.62 KB)

In [301]:
import tensorflow
opt = tensorflow.optimizers.Adam(0.1) #optimizer for training of model

In [302]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [303]:
#Setting up the tensorboard for better visualization
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [304]:
#Early stopping for best results
early_stop = EarlyStopping(monitor='val_loss', patience=150,restore_best_weights=True )

In [305]:
#fitting the model for initial accuracy
history = model.fit(
    X_train_under, y_train_under,
    validation_data=(X_test, y_test),
    epochs=150,
    batch_size=32,
    callbacks=[early_stop, tensorflow_callback],
    
)

Epoch 1/150
398/398 ━━━━━━━━━━━━━━━━━━━━ 1s 750us/step - accuracy: 0.7912 - loss: 0.4485 - val_accuracy: 0.8080 - val_loss: 0.4379
Epoch 2/150
398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.8008 - loss: 0.4438 - val_accuracy: 0.7810 - val_loss: 0.4625
Epoch 3/150
398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - accuracy: 0.7972 - loss: 0.4399 - val_accuracy: 0.7340 - val_loss: 0.5296
Epoch 4/150
398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.7932 - loss: 0.4555 - val_accuracy: 0.8035 - val_loss: 0.4116
Epoch 5/150
398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step - accuracy: 0.8099 - loss: 0.4223 - val_accuracy: 0.8410 - val_loss: 0.4001
Epoch 6/150
398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step - accuracy: 0.7945 - loss: 0.4410 - val_accuracy: 0.8420 - val_loss: 0.3969
Epoch 7/150
398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - accuracy: 0.8010 - loss: 0.4335 - val_accuracy: 0.8340 - val_loss: 0.3796
Epoch 8/150
398/398 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - accuracy: 0.7927 - loss: 0

In [306]:
model.save('model.h5')

In [307]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [308]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 60287), started 0:24:30 ago. (Use '!kill 60287' to kill it.)